In [8]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from math import exp
from google.colab import drive
drive.mount('/content/drive')

# Load data
file_path = '/content/drive/MyDrive/MATKUL/SEM 4/Optimasi AI/Mencoba Algoritma/train_2v.csv'
df = pd.read_csv(file_path).dropna()  # Drop rows with missing values

# Sample the dataset
SAMPLE_SIZE = 500
subset_idx = np.random.choice(len(df), SAMPLE_SIZE, replace=False)
df_sampled = df.iloc[subset_idx]

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [9]:
# Assuming the target label is the last column and all other columns are features
X = df_sampled.iloc[:, :-1].values  # Features (all columns except the last)
y = df_sampled.iloc[:, -1].values   # Target label (last column)

# Convert string columns to numerical using one-hot encoding
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
for i in range(X.shape[1]): # Loop through each column
    if isinstance(X[0, i], str): # Check if the column contains strings
        X[:, i] = label_encoder.fit_transform(X[:, i]) # Encode strings to numerical values

In [10]:
# Split dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [11]:
# Define Moth Flame Optimizer (MFO)
def moth_flame_optimizer(svm_model, X_train, y_train, X_test, y_test, max_iter=50, n_moths=50, lower_bound=-5, upper_bound=5):
    # Parameters for MFO
    best_accuracy = 0
    best_params = None

    # Moth initialization (random positions)
    positions = np.random.uniform(lower_bound, upper_bound, (n_moths, 2))  # C and gamma
    for iteration in range(max_iter):
        # Evaluate each moth (position)
        for i in range(n_moths):
            C = 10 ** positions[i, 0]
            gamma = 10 ** positions[i, 1]

            # Train the SVM with the current parameters
            svm_model.C = C
            svm_model.gamma = gamma
            svm_model.fit(X_train, y_train)

            # Predict and evaluate accuracy
            y_pred = svm_model.predict(X_test)
            accuracy = accuracy_score(y_test, y_pred)

            # Update the best solution
            if accuracy > best_accuracy:
                best_accuracy = accuracy
                best_params = (C, gamma)

        # Update positions based on best solution (flame update)
        positions = positions + np.random.rand(n_moths, 2) * (best_params - positions)

        # Enforce bounds
        positions = np.clip(positions, lower_bound, upper_bound)

    return best_params, best_accuracy

In [12]:
# Define the SVM model
svm_model = SVC(kernel='rbf')

In [13]:
# Apply Moth Flame Optimizer
best_params, best_accuracy = moth_flame_optimizer(svm_model, X_train, y_train, X_test, y_test)

In [14]:
# Output best hyperparameters and the resulting accuracy
print("Best Parameters: C =", best_params[0], "gamma =", best_params[1])
print("Best Accuracy:", best_accuracy)

Best Parameters: C = 26.48716700995068 gamma = 0.006502114159726931
Best Accuracy: 0.98
